In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, IntegerType
from pyspark.sql.functions import from_json, col, lit, from_unixtime
from pyspark.sql.functions import year, month, dayofmonth, hour

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-189047de-f1dc-45b5-afd9-17b042c2bd2e;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.5 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.5 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
	found org.apache.kafka#kafka-clients;3.9.1 in central
	found com.github.luben#zstd-jni;1.5.6-4 in central
	found org.l

In [3]:
kafka_options = {
    "kafka.bootstrap.servers": "kafka:9092",
    "subscribe": "wiki",
    "startingOffsets": "latest",
    "kafka.group.id": "wiki_group"
}

In [4]:
df = spark.readStream \
    .format("kafka") \
    .options(**kafka_options) \
    .load()

25/08/20 06:14:32 WARN KafkaSourceProvider: Kafka option 'kafka.group.id' has been set on this query, it is
 not recommended to set this option. This option is unsafe to use since multiple concurrent
 queries or sources using the same group id will interfere with each other as they are part
 of the same consumer group. Restarted queries may also suffer interference from the
 previous run having the same group id. The user should have only one query per group id,
 and/or set the option 'kafka.session.timeout.ms' to be very small so that the Kafka
 consumers from the previous query are marked dead by the Kafka group coordinator before the
 restarted query starts running.
    


In [5]:
schema = StructType([
    StructField("id", StringType()),
    StructField("wiki", StringType()),
    StructField("timestamp", IntegerType()),
    StructField("bot", BooleanType())
])
parsed_df = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")\
    .select(from_json(col("value"), schema).alias("data"))

In [6]:
final_df = parsed_df.select(
        col("data.id").alias("id"),
        col("data.wiki").alias("wiki"),
        col("data.timestamp").alias("timestamp"),
        col("data.bot").alias("bot")) \
    .withColumn("timestamp", from_unixtime(col("timestamp"))) \
    .withColumn("year", year(col("timestamp"))) \
    .withColumn("month", month(col("timestamp"))) \
    .withColumn("day", dayofmonth(col("timestamp"))) \
    .withColumn("hour", hour(col("timestamp")))

In [7]:
query = final_df.writeStream \
    .outputMode("append") \
    .format("parquet") \
    .option("path", "/datalake/data/wiki") \
    .option("checkpointLocation", "/datalake/checkpoint/wiki") \
    .partitionBy("year", "month", "day", "hour") \
    .trigger(processingTime="5 seconds") \
    .start()
query.awaitTermination()

25/08/20 06:14:38 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/08/20 06:14:38 WARN KafkaSourceProvider: Kafka option 'kafka.group.id' has been set on this query, it is
 not recommended to set this option. This option is unsafe to use since multiple concurrent
 queries or sources using the same group id will interfere with each other as they are part
 of the same consumer group. Restarted queries may also suffer interference from the
 previous run having the same group id. The user should have only one query per group id,
 and/or set the option 'kafka.session.timeout.ms' to be very small so that the Kafka
 consumers from the previous query are marked dead by the Kafka group coordinator before the
 restarted query starts running.
    
25/08/20 06:15:00 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 5764 milliseconds
25/08/20 06:17:38 WARN

StreamingQueryException: [STREAM_FAILED] Query [id = e76f7322-d183-4969-8d77-61e56997a745, runId = 5ecfc9db-4642-478d-9dff-122f1cc93182] terminated with exception: [INTERNAL_ERROR] The Spark SQL phase planning failed with an internal error. You hit a bug in Spark or the Spark plugins you use. Please, report this bug to the corresponding communities or vendors, and provide the full stack trace.